### ETL
El proceso ETL es un paso crucial para preparar los datos para su análisis. Al **extraer, transformar y cargar** los datos de forma adecuada, se obtienen datos de mejor calidad y se facilita la obtención de información valiosa a partir de ellos.



### Importamos librerías

Comienza instalando solo las librerías que necesitas para empezar tu proyecto.

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

Lectura del archivo **australian_users_items.parquet**

In [2]:
#la siguiente línea de código lee el archivo australian_users_items.parquet y lo convierte en un DataFrame de Pandas
items = pq.read_table("../0-DATA/australian_users_items.parquet").to_pandas()

In [3]:
#presentación en un DataFrame los datos, analizamos como está estructurado nuestros datos.
items.head(3)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [4]:
items.info() # no existe nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          int64 
 5   items_count       int64 
 6   user_id           object
 7   user_url          object
dtypes: int64(5), object(3)
memory usage: 314.5+ MB


### Transformación de nuestros Datos

Analizando columnas antes de eliminarlas

In [5]:
len(items["item_id"].unique()) # id de los videojuegos
#items["item_name"].unique() # nombre de los videojuegos
#items["playtime_forever"].unique() # tiempo de juego en minutos
#items["playtime_2weeks"].unique() # --> eliminar no aporta datos para nuestro proyecto
#items["steam_id"].unique() # --> eliminar por que tiene informacion similar a 'user_id'
#items["items_count"].unique() # tiene el numero de compras de cada usuario
#items["user_id"].unique() # id usuario
#items["user_url"].unique() # --> eliminamos por que son puras urls

10978

In [6]:
# Eliminando columnas
items = items.drop(columns=["user_url", "playtime_2weeks", "steam_id"])
items.head(3)

,item_id,item_name,playtime_forever,items_count,user_id
0,10,Counter-Strike,6,277,76561197970982479
1,20,Team Fortress Classic,0,277,76561197970982479
2,30,Day of Defeat,7,277,76561197970982479


Eliminación de valores duplicados

In [7]:
# analizaremos si tenemos filas duplicadas
duplicados = items.loc[items.duplicated()]
duplicados.shape #59117 filas duplicadas

(59117, 5)

In [8]:
#Eliminamos filas los duplicados, solo el 1.2% del total
items = items.drop_duplicates(keep='first')
items.shape

(5094092, 5)

In [9]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094092 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   playtime_forever  int64 
 3   items_count       int64 
 4   user_id           object
dtypes: int64(3), object(2)
memory usage: 233.2+ MB


### Modificando la columna 'playtime_forever' 

Columna **playtime_forever**

In [10]:
# Creamos la columna 'playtime_forever' donde guardaremos las horas de juego
items['playtime_hours'] = round(items['playtime_forever']/60,2)

Analizamos el tiempo de juego

In [11]:
#podemos ver que en 1847730 filas nos indican que el tiempo de juego es 0.
len(items[items['playtime_hours'] == 0])

1847730

In [12]:
#hacemos un filtro para eliminar todas las filas donde las horas de juegos es igual a 0
items = items[items['playtime_hours'] != 0]


In [13]:
#eliminamos la columna 'release_date', tenemos en su lugar la columna 'playtime_hours'
items = items.drop('playtime_forever', axis=1)
items.head(5)

,item_id,item_name,items_count,user_id,playtime_hours
0,10,Counter-Strike,277,76561197970982479,0.10
2,30,Day of Defeat,277,76561197970982479,0.12
8,300,Day of Defeat: Source,277,76561197970982479,78.88
9,240,Counter-Strike: Source,277,76561197970982479,30.88
10,3830,Psychonauts,277,76561197970982479,5.55


Guardamos el ETL el archivos CSV y Parquet

In [14]:
tabla = pa.Table.from_pandas(items) 
pq.write_table(tabla,"../0-DATA/australian_users_itemsETL.parquet") 

In [15]:
items.to_csv("../0-DATA/australian_users_itemsETL.csv", index=False, encoding="utf-8") 